In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype import config, logging
from datetime import date

In [ ]:
#Set variables
subject_list = ['sub-A200', 'sub-A201']

user = expanduser('~')

if user == '/Users/lucindasisk':
    home = join(user,'Box/LS_Folders/CANDLab_LS/DATA')
    workflow_dir = join(home, 'workflow_LS')
    data_dir = join(home, 'DataDir_LS')

In [ ]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(t1 = join(home,'{subject_id}/anat/{subject_id}_ses-shapesV1_T1w.nii.gz'),
                dti = join(home,'{subject_id}/dwi/{subject_id}_ses-shapesV1_dwi.nii.gz'),
                bval = join(home,'{subject_id}/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(home,'{subject_id}/dwi/{subject_id}_ses-shapesV1_dwi.bvec'),
                fmappa = join(home,'{subject_id}/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-PA_epi.nii.gz'),  
                fmapap = join(home,'{subject_id}/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-AP_epi.nii.gz'),
                aps = join(home, 'shapes_acqparams.txt'),
                index = join(home, 'shapes_index.txt')
               )#,
               # index = join(home, 'index.txt'),
               # aps = join(home, 'acqparams.txt'),
               # tryxfm = join(home, 'transform.txt'))

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

In [ ]:
#Eddy_CUDA Node
#FSL Eddy correction to remove eddy current distortion

eddy = Node(fsl.Eddy(is_shelled = True,
                    use_cuda=True,
                    cnr_maps=True,
                     repol=True),
           name = 'eddy')